In [3]:
import json
import copy

In [24]:
data_path = "../preprocess/data/result/"
hierarchy = json.load(open(data_path + 'AllTheNews/network/server/ravasz_hierarchies.json'))
level = 5

In [25]:
def getSubClusterNumDict(hierarchy, level):
        queue = copy.deepcopy(hierarchy['children'])
        sub_cluster_num_dict = {}
        while(len(queue) > 0):
            cur = queue[0]
            cur_level = int(cur['title'].split("-")[1])
            if cur_level == level:
                sub_cluster = cur['children']
                sub_cluster_ids = list(map(lambda cluster: cluster['title'], sub_cluster))
                sub_cluster_num_dict[cur['title'].split("-")[2]] = sub_cluster_ids
            elif cur_level > level:
                queue += cur['children']
            queue = queue[1:]
        return sub_cluster_num_dict

getSubClusterNumDict(hierarchy, level)

{'1': ['L-4-1'],
 '0': ['L-4-0'],
 '5': ['L-4-7', 'L-4-12', 'L-4-5', 'L-4-11', 'L-4-6', 'L-4-9', 'L-4-14'],
 '3': ['L-4-10', 'L-4-4', 'L-4-2', 'L-4-16', 'L-4-13'],
 '4': ['L-4-8', 'L-4-17', 'L-4-15', 'L-4-18'],
 '2': ['L-4-3']}

In [26]:
children = hierarchy['children']
ids = list(map(lambda child: child['title'], children))
print(ids)

['L-6-0', 'L-6-1']


In [4]:
media_sum = json.load(open('news_dialogue.json'))

In [5]:
len(media_sum)

463596

In [2]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [9]:
from pprint import pprint
pprint(media_sum[0])
subset = media_sum[0:100]
save_json(subset, 'subset.json')


{'date': '2007-11-28',
 'id': 'NPR-1',
 'program': 'News & Notes',
 'speaker': ['FARAI CHIDEYA, host',
             'FARAI CHIDEYA, host',
             'Mr. BLAIR UNDERWOOD (Actor)',
             'FARAI CHIDEYA, host',
             'FARAI CHIDEYA, host',
             'Ms. WENDY RAQUEL ROBINSON (Actress)',
             'Mr. KYLE BOWSER (Co-producer, "The Bible Experience: The '
             'Complete Bible")',
             'FARAI CHIDEYA, host',
             'Mr. KYLE BOWSER (Co-producer, "The Bible Experience: The '
             'Complete Bible")',
             'FARAI CHIDEYA, host',
             'Mr. KYLE BOWSER (Co-producer, "The Bible Experience: The '
             'Complete Bible")',
             'FARAI CHIDEYA, host',
             'FARAI CHIDEYA, host',
             'Ms. WENDY RAQUEL ROBINSON (Actress)',
             'Ms. WENDY RAQUEL ROBINSON (Actress)',
             'FARAI CHIDEYA, host',
             'Ms. WENDY RAQUEL ROBINSON (Actress)',
             'FARAI CHIDEYA, host',
   

In [16]:
# for interview in subset:
#     print(len(interview['utt']))
max_utt = max(map(lambda interview: len(interview['utt']), media_sum))
utts = list(map(lambda interview: len(interview['utt']), media_sum))

max_utt

AttributeError: 'list' object has no attribute 'indexOf'

In [17]:
print(utts.index(2827))

70436


In [19]:
pprint(media_sum[utts.index(2827)])
longest = media_sum[utts.index(2827)]
save_json(longest, 'longest.json')

{'date': '2000-12-2',
 'id': 'CNN-21017',
 'program': 'Special Event',
 'speaker': ['KYRA PHILLIPS, CNN ANCHOR',
             'BILL HEMMER, CNN ANCHOR',
             'MARK POTTER, CNN CORRESPONDENT',
             'HEMMER',
             'GRETA VAN SUSTEREN, CNN LEGAL ANALYST',
             'HEMMER',
             'GARY TUCHMAN, CNN CORRESPONDENT',
             'HEMMER',
             'TUCHMAN',
             'HEMMER',
             'TUCHMAN',
             'HEMMER',
             'TUCHMAN',
             'HEMMER',
             'TUCHMAN',
             'TUCHMAN',
             'HEMMER',
             'TUCHMAN',
             'HEMMER',
             'VAN SUSTEREN',
             'HEMMER',
             'VAN SUSTEREN',
             'TUCHMAN',
             'VAN SUSTEREN',
             'HEMMER',
             'VAN SUSTEREN',
             'HEMMER',
             'TUCHMAN',
             'HEMMER',
             'DAVID CARDWELL, CNN ELECTION LAW ANALYST',
             'HEMMER',
             'HEMMER',
           

In [11]:
import concurrent
from tqdm import tqdm
import copy
from pprint import pprint
import tiktoken
def request_chatgpt_gpt4(client, messages, format=None):
    model = 'gpt-3.5-turbo-0125'
    # model="gpt-4-1106-preview"
    if format == "json":
        response = client.chat.completions.create(
            # model="gpt-4-1106-preview",
            model = model,
            messages=messages,
            response_format={ "type": "json_object" },
            temperature=0,
        )
    else:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0,
        )
    return response.choices[0].message.content

def get_embedding(client, text, model="text-embedding-ada-002"):
    enc = tiktoken.encoding_for_model(model)
    print(len(enc.encode(text)))
    while len(enc.encode(text)) > 8191:
        text = text[:-100]
        print(len(enc.encode(text)))
    try:
        return client.embeddings.create(input = [text], model=model).data[0].embedding
    except Exception as e:
        print(e)
        return get_embedding(client, text, model)



def multithread_prompts(client, prompts):
    l = len(prompts)
    # results = np.zeros(l)
    with tqdm(total=l) as pbar:
        executor = concurrent.futures.ThreadPoolExecutor(max_workers=100)
        futures = [executor.submit(request_chatgpt_gpt4, client, prompt) for prompt in prompts]
        for _ in concurrent.futures.as_completed(futures):
            pbar.update(1)
    concurrent.futures.wait(futures)
    return [future.result() for future in futures]

def multithread_embeddings(client, texts):
    l = len(texts)
    # results = np.zeros(l)
    with tqdm(total=l) as pbar:
        executor = concurrent.futures.ThreadPoolExecutor(max_workers=100)
        futures = [executor.submit(get_embedding, client, text) for text in texts]
        for _ in concurrent.futures.as_completed(futures):
            pbar.update(1)
    concurrent.futures.wait(futures)
    return [future.result() for future in futures]
    return

def prompt_factory(entity):
    messages = [
        {
            "role": "system",
            "content": """You are a research assistant. The user needs you to explain some terminologies.  """ },
        {
            "role": "user",
            "content": "What does {} mean?".format(entity)
        }
    ]
    return messages


In [9]:
from openai import OpenAI
import json
openai_api_key = open("openai_api_key").read()
client=OpenAI(api_key=openai_api_key)
dataset = json.load(open("missed.json"))['missed']
print(dataset)
prompts = [prompt_factory(entity) for entity in dataset]
response = multithread_prompts(client, prompts)
result = []
for entity, response in zip(dataset, response):
    result.append({
        "entity": entity,
        "explanation": response
    })
save_json(result, 'missed_explanations.json')

['usability evalution', 'log file analysis', 'web proxy', 'webquilt', 'different fitness level', 'exertion experience', 'different physical ability', 'design tactic', 'balanced exertion', 'balanced experience', 'specific design', 'balanced exertion experience', 'sport', 'dynamic game', 'design space analysis', 'history mechanism', 'design decision', 'history log', 'history session', 'tools record', 'graphical history tool', 'graphical histories', 'interactive history tool', 'interaction history', 'database management systems', 'indexing terms', 'visual system', 'visualized data', 'large data set', 'data type', 'use data reduction method', 'data processing', 'data analyst', 'real-time visual querying', 'data size', 'multivariate data tile', 'developer tool', 'd3 data-driven documents', 'immediate evaluation', 'comparable notational efficiency', 'bind input data', 'intermediate representation', 'direct inspection', 'dramatic performance improvement', 'data-driven documents', 'standard do

100%|██████████| 1470/1470 [01:08<00:00, 21.41it/s]


In [12]:
entity_w_explanations = json.load(open("missed_explanations.json"))
explanations= list(map(lambda x: x['explanation'], entity_w_explanations))
embeddings = multithread_embeddings(client, explanations)
for embeddings, entity_w_explanation in zip(embeddings, entity_w_explanations):
    entity_w_explanation['embedding'] = embeddings
save_json(entity_w_explanations, 'missed_embeddings.json')


  0%|          | 0/1470 [00:00<?, ?it/s]

11689
105
117
98
100
87
86
135
114
128
79
136
65
104

95
94
100
77
89
109
79
90
89
103
99
116
96
106
98
93
94
113
60
76
132
65
102
67
86
96
68
101
90
88
84
92
99
127
175
129
116
104
120
86
176
168
92
98
153
70
81
108
103
88
174
80
117
104
96
81
92
111
80
100
104
83
121
77
91
98
73
97
101
82
107
93
91
76
101
77
82
86
137
63
88
116
107
86
124


  0%|          | 3/1470 [00:16<1:41:59,  4.17s/it]

169
103
95
112
152
120


  1%|          | 9/1470 [00:16<25:11,  1.03s/it]  

...

 92%|█████████▏| 1359/1470 [01:44<00:05, 20.04it/s]

132
102
90
74
141
91
110


 93%|█████████▎| 1367/1470 [01:45<00:05, 20.54it/s]

109
103
96
109
88
91
175
127


100%|██████████| 1470/1470 [02:27<00:00,  9.96it/s]
